# 1. What is PCA?
* The PCA Algorithm works by computing principal components where it retains datasets in the **direction of maximum variance** in the original datasets
* PCA is a procedure that uses an orthogonal transformation to convert a set of observations possibly correlated variables into a set of values of linearly uncorrelated variables.

## Import required libraries & data

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(42)

In [ ]:
data = pd.read_csv("activity_sleep_label.csv")
data

# 2. What is the step-by-step procedure of PCA?

## 2.1. Perform data pre-processing
(e.g. scaling on D-dimensional data)
* It is critical to perform normalization prior to implementing the PCA algorithm

In [ ]:
# Exclude categorical data (dtype = string)
to_be_normed = [col for col in data.columns if data[col].dtypes != 'object']
# 1. Standardize the numerical data
standardized_data = normalize(data[to_be_normed])
standardized_data

## 2.2. Compute a covariance matrix
* This is needed to understand how features are correlated with each other
* The covariance matrix is N X N symmetric matrix where N is the # of dimensions

In [ ]:
# 2. Calculate the covariance matrix
cov_matrix = np.cov(standardized_data, rowvar=False)
cov_matrix

In [ ]:
# The covariance matrix is N X N symmetric matrix where N is the # of dimensions
print(f"Dimension of the data:{standardized_data.shape[1]}")
print(f"Shape of the covariance matrix:{cov_matrix.shape}")

## 2.3. Calculate the eigenvalues and eigenvectors of the covariance matrix
* This is needed to determine the **principal components** of the dataset

In [ ]:
# 3. Calculate the eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
print(f"Dimension of the data: {standardized_data.shape[1]}")
print(f"The number of the eigenvectors: {len(eigenvectors)}")
print(f"Shape of the eigenvectors: {eigenvectors.shape}")

In [ ]:
eigenvalues

In [ ]:
eigenvectors

## 2.4. Sort the eigenvalue in descending order and find the corresponding eigenvector.

In [ ]:
# 4. Sort eigenvectors in descending order of eigenvalues
sorted_indices = np.argsort(eigenvalues)[::-1]
sorted_eigenvalue = eigenvalues[sorted_indices]
sorted_eigenvectors = eigenvectors[:,sorted_indices]

In [ ]:
sorted_eigenvalue

In [ ]:
sorted_eigenvectors = eigenvectors[:,sorted_indices]
sorted_eigenvectors

## 2.5. Select the largest K cases (**K <= D**) and create a projection matrix (i.e. w) using the cases
* K refers to the number of principal components
* How to create a projection matrix?
  * If K=2, select two corresponding eigenvectors identified by the sorted eigenvalues
* How to determine the number of principal components?
  * Scree plot: a line plot of the eigenvalues of the eigenvalues of principal components

In [ ]:
# 5. Select the largest 'num_components' eigenvectors
num_components = 2
principal_components = eigenvectors[:, 0:num_components]

In [ ]:
principal_components

## 2.6. Transform data into a lower-dimensional space using the projection matrix
* For two principal components, we have:
  * Transforming by **dot product** (normalized data, projection matrix)
    * The dot product of the normalized data matrix and the projection matrix effectively projects the original data onto the subspace defined by the principal components.

In [ ]:
# 6. Project the data onto the principal components
transformed_data = np.dot(principal_components.transpose(), standardized_data.transpose()).transpose()

In [ ]:
transformed_data.shape

# 3. Let's Compare classification w/w.o PCA (Preparation & functions)

## 3.0. Define a PCA function before the application

In [ ]:
def pca(data, num_component=2, threshold = 0.01, scree = True):
    # Exclude categorical data (dtype = string)
    # to_be_normed = [col for col in X.columns if X[col].dtypes != 'object']
    # 1. Standardize the numerical data
    standardized_data = normalize(data)

    # 2. Calculate the covariance matrix
    cov_matrix = np.cov(standardized_data, rowvar=False)

    # 3. Calculate the eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

    # 4. Sort eigenvectors in descending order of eigenvalues
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvalue = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:,sorted_indices]

    # 5. Select the largest 'num_components' eigenvectors
    principal_components = sorted_eigenvectors[:, 0:num_component]

    # 6. Project the data onto the principal components
    transformed_data = np.dot(principal_components.transpose(), standardized_data.transpose()).transpose()

    # 7. Calculate the explained variance ratio
    explained_variance_ratio = [(i/sum(eigenvalues)) for i in sorted_eigenvalue]

    if scree == True:
        scree_plot(explained_variance_ratio, num_component, threshold)
    
    return transformed_data

## 3.1. How would you determine the number of principal components?
### Scree Plot
* A scree plot is a line of the eigenvalues of principal components

In [ ]:
def scree_plot(explained_variance_ratio, num_component, threshold):
    num_components = len(explained_variance_ratio)

    fig = plt.figure(figsize=(20,6))
    ax = fig.add_subplot()

    cumulated = np.cumsum(explained_variance_ratio)
    ax.bar(range(1, num_components +1), explained_variance_ratio, color = '#99ccff')
    ax.plot(range(1, num_components+1), cumulated, color = 'black')
    
    max_val = 0
    
    for value in cumulated:
        if (value - max_val) >= threshold:
            max_val = value
    
    ax.vlines(num_component, 0, max(cumulated), color = "blue", linestyles="--",label="your PC #")
    ax.vlines(np.where(cumulated==max_val)[0]+1, 0, max(cumulated), color = "red", linestyles="--", label = "Best PC #")
    
    for i in range(num_components): 
        ax.annotate(r"%s" % ((str(explained_variance_ratio[i]*100)[:4])), (i+1, explained_variance_ratio[i]), va = "bottom", ha = "center")
     
    ax.set_xticks(np.arange(1,num_components+1))
    ax.set_xlabel("Principal Components", fontsize = 18)
    ax.set_ylabel("Explained variance Ratio", fontsize = 18)
    
    plt.legend(loc = "best", fontsize = 15)
    plt.title('Scree plot - PCA Scratch', fontsize = 20)

## 3.2. Data Split

In [ ]:
X = data.iloc[:,2:].to_numpy()
y = data.iloc[:, 1].to_numpy()

In [ ]:
# 전체 데이터 세트를 학습 세트(training set)와 검증 세트(test set)로 나눔
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y,random_state=42)
print(len(X_train), len(X_test))

## 3.3. Data w/w.o. PCA
### Without PCA (Just standardization)

In [ ]:
# Z-score 표준화: 평균을 0, 표준편차 1로 변환
scaler = StandardScaler() # Scaler 객체 생성
scaler.fit(X_train) # 스케일링(표준화)를 위한 평균과 표준 편차 계산
X_train_stand = scaler.transform(X_train) # 스케일링(표준화 수행)
X_test_stand = scaler.transform(X_test)

### With PCA (PC = 2)

In [ ]:
X_train_pca_2 = pca(X_train, 2, 0.01 , scree = True)

In [ ]:
X_test_pca_2 = pca(X_test, 2, 0.01, scree = False)

### With PCA (PC = 7)

In [ ]:
X_train_pca_7 = pca(X_train, 7,0.01)

In [ ]:
X_test_pca_7 = pca(X_test, 7,0.01, scree = False)

## 3.4. Scratch vs library (scree plot)

In [ ]:
# Normalize the Data
X_train_norm = normalize(X_train)
X_test_norm = normalize(X_test)

In [ ]:
pca = PCA()
X_train_pca_lib = pca.fit_transform(X_train_norm)
explained_variance_ratio = pca.explained_variance_ratio_

In [ ]:
def lib_scree_plot(pca_instance, threshold = 0.01):
    num_components = len(pca_instance.explained_variance_ratio_)
    ind = np.arange(num_components)
    vals = pca_instance.explained_variance_ratio_ 
    
    fig = plt.figure(figsize=(20,6))
    ax = fig.add_subplot()
    cumvals = np.cumsum(vals)
    
    max_val = 0

    for value in cumulated:
        if (value - max_val) >= threshold:
            max_val = value
    
    ax.vlines(pca.n_components_, 0, max(cumulated), color = "blue", linestyles="--",label="your PC #")
    ax.vlines(np.where(cumulated==max_val)[0]+1, 0, max(cumulated), color = "red", linestyles="--", label = "Best PC #")
    
    ax.bar(ind, vals, color = '#ff6f15') # Bar plot
    ax.plot(ind, cumvals, color = 'black') # Line plot 


    for i in range(num_components): #라벨링(바 위에 텍스트(annotation) 쓰기)
        ax.annotate(r"%s" % ((str(vals[i]*100)[:3])), (ind[i], vals[i]), va = "bottom", ha = "center", fontsize = 13)
    
    ax.set_xticks(np.arange(1,num_components+1))
    ax.set_xlabel("Principal Components", fontsize = 18)
    ax.set_ylabel("Explained variance Ratio", fontsize = 18)

    plt.legend(loc = "best", fontsize = 15)
    plt.title('Scree plot - sklearn.decomposition PCA', fontsize = 20)

In [ ]:
lib_scree_plot(pca)

## 3.5. (K-NN classification) Determination of the optimal K-values of K-NN algorithm.

In [ ]:
def optimal_k(X_train, y_train, X_test, y_test, max_k = 30):
    errors = []
    for i in range(1, max_k+1):
        knn = KNeighborsClassifier(n_neighbors = i)
        knn.fit(X_train, y_train)
        pred_i = knn.predict(X_test)
        errors.append(np.mean(pred_i != y_test))

    opt_k = errors.index(min(errors)) + 1
    
    plt.plot(range(1, max_k+1), errors, marker='o')
    plt.vlines(opt_k, 0, max(errors), color = "red", linestyles="--")
    plt.title('Mean error depending on K-Value')
    plt.xlabel('k-value')
    plt.xticks(np.arange(1, max_k+1))
    plt.ylabel('mean error')
    plt.figure(figsize=(20,6))
    plt.show()
    
    return opt_k

## 3.6. Plot the Performance

### Plot confusion matrix

In [ ]:
# plot the performance of the model
def confusion_plot(conf_matrix, clf, title):
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=clf.classes_, yticklabels=clf.classes_)
    plt.title(f'Confusion Matrix-{title}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

### Plot accuracy plot according to number of neighborhood (K)

In [ ]:
# Return the list of the accuracy according to K-values for K-NN algorithm (for scratch PCA)
def acc_k_val(X_train, y_train, X_test, y_test, max_k = 30):
    accs = []
    for i in range(1, max_k+1):
        knn = KNeighborsClassifier(n_neighbors = i)
        knn.fit(X_train, y_train)
        pred_i = knn.predict(X_test)
        acc = accuracy_score(y_test, pred_i)
        accs.append(acc)

    return accs

In [ ]:
# Return the list of the accuracy according to K-values for K-NN algorithm (for library PCA)
def acc_k_val_lib(X_train, y_train, X_test, y_test, n_comp ,max_k = 30):
    accs = []
    for i in range(1, max_k+1):
        pca_lib = PCA(n_components = n_comp)
        X_train_pca_lib = pca_lib.fit_transform(X_train)
        X_test_pca_lib = pca_lib.fit_transform(X_test)
        knn = KNeighborsClassifier(n_neighbors = i)
        knn.fit(X_train_pca_lib, y_train)
        pred_i = knn.predict(X_test_pca_lib)
        acc = accuracy_score(y_test, pred_i)
        accs.append(acc)

    return accs

### Plot Accuracy plot according to the number of components of PCA

In [ ]:
# Return the list of accuracies according to the number of components
def acc_component_num(X_train, y_train, X_test, y_test, k_num):
    accs = []
    max_num = X_train.shape[1]
    for i in range(0, max_num+1):
        if i == 0:
            reduced_train = X_train
            reduced_test = X_test
        else:
            reduced_train = pca(X_train, i, scree = False)
            reduced_test = pca(X_test, i, scree=False)
        
        knn = KNeighborsClassifier(n_neighbors = k_num)
        knn.fit(reduced_train, y_train)
        pred_i = knn.predict(reduced_test)
        acc = accuracy_score(y_test, pred_i)
        accs.append(acc)

    return accs

# 4. K-NN Classifier


## 4.1. Classification without PCA

* pca를 쓰지 않기 때문에 standarzation을 따로 시켜줘야 한다.

In [ ]:
# Find optimal k value for K-NN classifier
opt_k = optimal_k(X_train_stand, y_train, X_test_stand, y_test, max_k = 30)

In [ ]:
# k-NN 분류기를 생성
classifier = KNeighborsClassifier(n_neighbors=opt_k)
# 분류기 학습
classifier.fit(X_train_stand, y_train)

In [ ]:
# 예측
y_pred_stand= classifier.predict(X_test_stand)

### 4.1.1. Result of classification without PCA

In [ ]:
no_pca_conf_matrix= confusion_matrix(y_test, y_pred_stand)
confusion_plot(no_pca_conf_matrix, classifier, "without PCA")

In [ ]:
no_pca_performance = classification_report(y_test, y_pred_stand)
print(no_pca_performance)

## 4.2. Classification with PCA

### 4.2.1. When PC = 2

In [ ]:
# Find optimal k value for K-NN classifier
opt_k2 = optimal_k(X_train_pca_2, y_train, X_test_pca_2, y_test, max_k = 30)

In [ ]:
# k-NN 분류기를 생성
classifier = KNeighborsClassifier(n_neighbors=opt_k2)
# 분류기 학습
classifier.fit(X_train_pca_2, y_train)

In [ ]:
# 예측
y_pred_pca2= classifier.predict(X_test_pca_2)

In [ ]:
pca2_conf_matrix= confusion_matrix(y_test, y_pred_pca2)
confusion_plot(pca2_conf_matrix, classifier, "with PCA(PC=2)")

In [ ]:
pca2_performance = classification_report(y_test, y_pred_pca2)
print(pca2_performance)

### 4.2.2. When PC = 6 (Best # of PC)

In [ ]:
# Find optimal k value for K-NN classifier
opt_k7 = optimal_k(X_train_pca_7, y_train, X_test_pca_7, y_test, max_k = 30)

In [ ]:
# k-NN 분류기를 생성
classifier = KNeighborsClassifier(n_neighbors=opt_k7)
# 분류기 학습
classifier.fit(X_train_pca_7, y_train)

In [ ]:
# 예측
y_pred_pca7= classifier.predict(X_test_pca_7)

In [ ]:
pca7_conf_matrix= confusion_matrix(y_test, y_pred_pca7)
confusion_plot(pca7_conf_matrix, classifier, "with PCA(PC = 7)")

In [ ]:
pca7_performance = classification_report(y_test, y_pred_pca7)
print(pca7_performance)

* Same Performance

## 4.3. Accuracy of K-NN classification according to K values

In [ ]:
k = 30

acc_no = acc_k_val(X_train_stand, y_train, X_test_stand, y_test, k)
acc_2 = acc_k_val(X_train_pca_2, y_train, X_test_pca_2, y_test, k)
acc_7 = acc_k_val(X_train_pca_7, y_train, X_test_pca_7, y_test, k)

plt.plot(range(1, k+1), acc_no, marker='o', label = "K-NN")
plt.plot(range(1, k+1), acc_2, marker='o', label = "K-NN + PCA(2)")
plt.plot(range(1, k+1), acc_7, marker='o', label = "K-NN + PCA(7)")
plt.title('Accuracy of K-NN depending on K-Value (Scratch PCA)')
plt.xlabel('k-value')
plt.xticks(np.arange(1, k+1))
plt.ylabel('Accuracy')
plt.yticks(np.arange(0.20, 0.75, 0.05))
plt.legend(loc="lower right")
plt.figure(figsize=(20,6))


plt.show()

In [ ]:
X_train_norm = normalize(X_train)
X_test_norm = normalize(X_test)

In [ ]:
k = 30

acc_2_lib = acc_k_val_lib(X_train_norm, y_train, X_test_norm, y_test, 2, k)
acc_7_lib = acc_k_val_lib(X_train_norm, y_train, X_test_norm, y_test, 7, k)

plt.plot(range(1, k+1), acc_no, marker='o', label = "K-NN")
plt.plot(range(1, k+1), acc_2_lib, marker='o', label = "K-NN + PCA(2)")
plt.plot(range(1, k+1), acc_7_lib, marker='o', label = "K-NN + PCA(7)")
plt.title('Accuracy of K-NN depending on K-Value (sklearn.decomposition PCA)')
plt.xlabel('k-value')
plt.xticks(np.arange(1, k+1))
plt.ylabel('Accuracy')
plt.yticks(np.arange(0.20, 0.75, 0.05))
plt.legend(loc="lower right")
plt.figure(figsize=(20,6))


plt.show()

## 4.4. Accuracy according to the number of Principal component (k = 22)

In [ ]:
X_train.shape[1]

In [ ]:
acc_to_pca_6 = acc_component_num(X_train, y_train, X_test, y_test, 6)
acc_to_pca_15 = acc_component_num(X_train, y_train, X_test, y_test, 15)
acc_to_pca_22 = acc_component_num(X_train, y_train, X_test, y_test, 22)

plt.figure(figsize=(25,6))

plt.plot(range(0, X_train.shape[1]+1), acc_to_pca_6, marker='o', label = "KNN")
plt.plot(range(0, X_train.shape[1]+1), acc_to_pca_15, marker='o', label = "KNN+PCA(PC=2)")
plt.plot(range(0, X_train.shape[1]+1), acc_to_pca_22, marker='o', label = "KNN+PCA(PC=7)")
plt.title(f'Accuracy of K-NN depending on number of components for PCA', fontsize=18)
plt.xlabel('num of pca', fontsize = 15)
plt.xticks(np.arange(0, X_train.shape[1]+1), fontsize = 12)
plt.ylabel('Accuracy', fontsize = 15)

plt.legend(loc = "best")

plt.show()

# Reference
1. [PCA scratch](https://www.askpython.com/python/examples/principal-component-analysis)
2. [Data Normalization](https://stackoverflow.com/questions/21030391/how-to-normalize-a-numpy-array-to-a-unit-vector)
3. [Scree plot - concept](https://sanchitamangale12.medium.com/scree-plot-733ed72c8608)
4. [Scree plot - implementation](https://velog.io/@yuns_u/PCA-Scree-Plot)
5. [K-NN Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)
6. [Debugging and visualization](https://chat.openai.com)